In [3]:
rm(list=ls())

library(DoubleML)
library(ggplot2)
library(mlr3)
library(mlr3learners)
library(data.table)
library(mvtnorm)
library(checkmate)
library(glmnet)
library(ggplot2)
library(graphics)
library(tidyverse)
library(haven)
#install.packages("sjlabelled")
library(sjlabelled)
#install.packages("varhandle")
library(varhandle)
#install.packages("stargazer")
library(stargazer)
#install.packages("xtable")
library(xtable)
suppressMessages(library(parallel))
suppressMessages(library(ggplot2))
#suppressMessages(library(doMC))
lgr::get_logger("mlr3")$set_threshold("warn")

#numCores <- detectCores()
#registerDoMC(cores = numCores)

Warning message:
"package 'DoubleML' was built under R version 4.1.2"
Warning message:
"package 'mlr3' was built under R version 4.1.2"
Warning message:
"package 'mlr3learners' was built under R version 4.1.2"
Warning message:
"package 'data.table' was built under R version 4.1.2"
Warning message:
"package 'mvtnorm' was built under R version 4.1.1"
Warning message:
"package 'checkmate' was built under R version 4.1.2"
Warning message:
"package 'glmnet' was built under R version 4.1.2"
Loading required package: Matrix

Loaded glmnet 4.1-3

Warning message:
"package 'tidyverse' was built under R version 4.1.1"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v tibble  3.1.2     v dplyr   1.0.7
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1
v purrr   0.3.4     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::be

# Machine Learning Estimation Functions

In [4]:
# Random Forest Estimation Function
est_forest <- function(obj_dml_data) {
    
    learner <- lrn("regr.ranger", num.trees=100, min.node.size=2, max.depth=5)
    ml_m <- learner$clone()
    ml_g <- learner$clone()
    
    obj_dml_plr  <- DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)
    
    obj_dml_plr$fit()
    theta = rbind(obj_dml_plr$coef,obj_dml_plr$se)
    return(theta)
}

# LASSO Estimation Function
est_lasso <- function(obj_dml_data) {
    
    learner <- lrn("regr.cv_glmnet", s="lambda.min")
    ml_g <- learner$clone()
    ml_m <- learner$clone()

    obj_dml_plr  <- DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)

    obj_dml_plr$fit()
    theta = rbind(obj_dml_plr$coef, obj_dml_plr$se)
    return(theta)
}

# Regression Tree Estimation
est_rt   <- function(obj_dml_data){

    learner <- lrn("regr.rpart")
    ml_g <- learner$clone()
    ml_m <- learner$clone()
    
    obj_dml_plr = DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)

    param_grid = list(
      "ml_g" = paradox::ParamSet$new(list(
            paradox::ParamDbl$new("cp", lower = 0.01, upper = 0.02),
            paradox::ParamInt$new("minsplit", lower = 1, upper = 2))),
      "ml_m" = paradox::ParamSet$new(list(
            paradox::ParamDbl$new("cp", lower = 0.01, upper = 0.02),
            paradox::ParamInt$new("minsplit", lower = 1, upper = 2))))

    # minimum requirements for tune_settings
    tune_settings = list(
    terminator = mlr3tuning::trm("evals", n_evals = 5),
    algorithm = mlr3tuning::tnr("grid_search", resolution = 5))
    obj_dml_plr$tune(param_set = param_grid, tune_settings = tune_settings)
    obj_dml_plr$fit()
    
    theta = rbind(obj_dml_plr$coef,obj_dml_plr$se)
    return(theta)

}


In [7]:
#df<-pdata.frame(panel_data_set,index=c("village",))

###function input is a df of control variables
lasso_data_generator<-function(df){
    len<-ncol(df)
##each variable is multiplied by all other variables so includes
    repeat_remover<-0
    for (variable in 1:len){
        for (loop in 1:len){
            if(loop + repeat_remover <=len){
            df=cbind(df,(df[,variable])*(df[,loop+repeat_remover]))
 
            }
        }
    repeat_remover<- repeat_remover+1
    }
 return(df)
}
                     
add_fixed_effect<-function(df, variable){
    df<-cbind(df,to.dummy(df$variable, "dummy"))
    return(df)
}


In [10]:
setwd("
df<-read_dta("merged_and_edited_stata.dta")


ERROR: Error: 'merged_and_edited_stata.dta' does not exist in current working directory ('C:/Users/hugo_/Documents/Econometrics-RM-main/Econometrics-RM-main').


In [8]:
Y_var= c("balance_final", "balance_average", "exp_freq", "exp_tempt", "sav_cash", "sav_nonbcsa_all", "sav_bcsa_all")
control=c("cat_bcsa_open", "female", "sc", "obc", "fc", "readwrite_dum", "married", "age", "emp_agr", "emp_nonagr", "self_agr", "self_nonagr", "land_amount", "dwelling_katcha", "account_total", "shg_total", "savings_decision", "trust_bank_bcsa", "impatient", "bcsa_distance","village")
regression_df<-data.frame(matrix(ncol=2,nrow=442))
colnames(regression_df)<-c("y","d")
regression_df["d"]<-df["random_account"]
theta<-data.frame(matrix(ncol=7,nrow=6))
colnames(theta)<-c("balance_final", "balance_average", "exp_freq", "exp_tempt", "sav_cash", "sav_nonbcsa_all", "sav_bcsa_all")

regressed<-c()
for (var in Y_var){
    regression_df["y"]<-df[,var]  
    test_data<-subset(df,select=c(cat_bcsa_open, female, sc, obc, fc, readwrite_dum, married, age, emp_agr, emp_nonagr, self_agr, self_nonagr, land_amount, dwelling_katcha, account_total, shg_total, savings_decision, trust_bank_bcsa, impatient, bcsa_distance, village))
    regressed<-c()
    regressed<-cbind(regression_df,test_data)
    regressed<-remove_all_labels(regressed)
        if (var != "balance_final"){
            if (var != "balance_average"){
                #regressed<-regressed[complete.cases(regressed),]
                #regressed<- regressed %>% drop_na(y,cat_bcsa_open, female, sc, obc, fc, readwrite_dum, married, age, emp_agr, emp_nonagr, self_agr, self_nonagr, land_amount, dwelling_katcha, account_total, shg_total, savings_decision, trust_bank_bcsa, impatient, bcsa_distance, village)
                regressed<- regressed %>% drop_na(y)

                #ompleterecords <- na.omit(regressed) 
            }
        }
        
    #regressed<-add_fixed_effect(regressed, village)
    regressed<-cbind(regressed, to.dummy(regressed$village, "dummy"))

    obj_dml_data <- double_ml_data_from_data_frame(regressed, y_col = "y", d_cols = "d")

    
    theta_forest <- est_forest(obj_dml_data)  
    theta_rt     <- est_rt(obj_dml_data)
    ###implement special data process for lasso
    test_data<-lasso_data_generator(test_data)
    regressed<-c()
    regressed<-cbind(regression_df,test_data)
    regressed<-remove_all_labels(regressed)
           if (var != "balance_final"){
            if (var != "balance_average"){
                #regressed<-regressed[complete.cases(regressed),]
                #regressed<- regressed %>% drop_na(y,cat_bcsa_open, female, sc, obc, fc, readwrite_dum, married, age, emp_agr, emp_nonagr, self_agr, self_nonagr, land_amount, dwelling_katcha, account_total, shg_total, savings_decision, trust_bank_bcsa, impatient, bcsa_distance, village)
                regressed<- regressed %>% drop_na(y)

                #ompleterecords <- na.omit(regressed) 
            }
        }
        
    #regressed<-add_fixed_effect(regressed, village)
    regressed<-cbind(regressed, to.dummy(regressed$village, "dummy"))
    
    obj_dml_data <- double_ml_data_from_data_frame(regressed, y_col = "y", d_cols = "d")
    theta_lasso  <- est_lasso(obj_dml_data)
    theta[,var] <- rbind(theta_lasso, theta_forest, theta_rt)
    #thetas$var<-theta
    regressed<-c()
    
}

INFO  [11:33:08.235] [bbotk] Starting to optimize 2 parameter(s) with '<OptimizerGridSearch>' and '<TerminatorEvals> [n_evals=5, k=0]' 
INFO  [11:33:08.251] [bbotk] Evaluating 1 configuration(s) 
INFO  [11:33:08.466] [bbotk] Result of batch 1: 
INFO  [11:33:08.468] [bbotk]      cp minsplit regr.mse runtime_learners                                uhash 
INFO  [11:33:08.468] [bbotk]  0.0125        2  1022372             0.08 d8b2fb9e-b1c3-4086-9581-0776cc688416 
INFO  [11:33:08.471] [bbotk] Evaluating 1 configuration(s) 
INFO  [11:33:08.680] [bbotk] Result of batch 2: 
INFO  [11:33:08.696] [bbotk]      cp minsplit regr.mse runtime_learners                                uhash 
INFO  [11:33:08.696] [bbotk]  0.0125        1 811532.9             0.09 a671a812-c0ff-400f-9eab-7f12851e3ccd 
INFO  [11:33:08.696] [bbotk] Evaluating 1 configuration(s) 
INFO  [11:33:08.907] [bbotk] Result of batch 3: 
INFO  [11:33:08.907] [bbotk]    cp minsplit regr.mse runtime_learners                            

In [ ]:
print(xtable(theta, type = "latex"), file = "filename2.tex")


In [9]:
print(theta)

          d         d         d         d         d        d        d
1 465.84841 318.44800 -438.8547  2.199927 -329.4826 617.8294 933.2167
2  77.96585  68.56621  208.1169 63.887377  493.0996 570.6952 568.5677
3 450.66216 309.91050 -364.1761 40.580599 -174.6768 326.7524 918.0017
4  75.91422  64.72746  203.1010 59.373258  457.4002 520.9940 521.0881
5 475.03463 314.86205 -381.8007 19.732819 -226.0369 370.4904 890.8066
6  83.20856  71.10903  223.4133 66.109631  479.9026 563.1179 573.5339
